In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress

# Range of latitudes and longitudes
lat_range = (-90, 90)
lon_range = (-180, 180)

# Files to load
file_to_load = "Resources/bfro_reports_geocoded.csv"

# Read bigfoot sightings file and store into Pandas data frame
bigfoot_data = pd.read_csv(file_to_load)

bigfoot_data.head()

,observed,location_details,county,state,season,title,latitude,longitude,date,number,...,moon_phase,precip_intensity,precip_probability,precip_type,pressure,summary,uv_index,visibility,wind_bearing,wind_speed
0,Ed L. was salmon fishing with a companion in P...,East side of Prince William Sound,Valdez-Chitina-Whittier County,Alaska,Fall,NaN,NaN,NaN,NaN,1261.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,heh i kinda feel a little dumb that im reporti...,"the road is off us rt 80, i dont know the exit...",Warren County,New Jersey,Fall,NaN,NaN,NaN,NaN,438.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I was on my way to Claremont from Lebanon on R...,Close to Claremont down 120 not far from Kings...,Sullivan County,New Hampshire,Summer,Report 55269: Dawn sighting at Stevens Brook o...,43.41549,-72.33093,2016-06-07,55269.0,...,0.10,0.001,0.7,rain,998.87,Mostly cloudy throughout the day.,6.0,9.70,262.0,0.49
3,I was northeast of Macy Nebraska along the Mis...,Latitude & Longitude : 42.158230 -96.344197,Thurston County,Nebraska,Spring,Report 59757: Possible daylight sighting of a ...,42.15685,-96.34203,2018-05-25,59757.0,...,0.38,0.000,0.0,NaN,1008.07,Partly cloudy in the morning.,10.0,8.25,193.0,3.33
4,"While this incident occurred a long time ago, ...","Ward County, Just outside of a the Minuteman T...",Ward County,North Dakota,Spring,Report 751: Hunter describes described being s...,48.25422,-101.31660,2000-04-21,751.0,...,0.60,NaN,NaN,rain,1011.47,Partly cloudy until evening.,6.0,10.00,237.0,11.14


In [2]:
# Read over the data and get the statistics on latitudes and longitudes

# Latitude variables
lat_avg = bigfoot_data["latitude"].mean()
lat_med = bigfoot_data["latitude"].median()
lat_mode = bigfoot_data["latitude"].mode()
lat_mode = ["%.2f" % elem for elem in lat_mode]
lat_min = bigfoot_data["latitude"].min()
lat_max = bigfoot_data["latitude"].max()

# Longitude variables
lon_avg = bigfoot_data["longitude"].mean()
lon_med = bigfoot_data["longitude"].median()
lon_mode = bigfoot_data["longitude"].mode()
lon_mode = ["%.2f" % elem for elem in lon_mode]
lon_min = bigfoot_data["longitude"].min()
lon_max = bigfoot_data["longitude"].max()

# Create summary table
summary_lat_lon = pd.DataFrame({"Lat/Lon": ["Latitude", "Longitude"],
                               "Average": [lat_avg, lon_avg],
                               "Median": [lat_med, lon_med],
                               "Min": [lat_min, lon_min],
                               "Max": [lat_max, lon_max],
                               "Mode": [[lat_mode], [lon_mode]]})

# Rounding the values to 2 decimals
summary_lat_lon["Average"] = summary_lat_lon["Average"].astype(float).map("{:,.2f}".format)
summary_lat_lon["Median"] = summary_lat_lon["Median"].astype(float).map("{:,.2f}".format)
summary_lat_lon["Min"] = summary_lat_lon["Min"].astype(float).map("{:,.2f}".format)
summary_lat_lon["Max"] = summary_lat_lon["Max"].astype(float).map("{:,.2f}".format)

summary_lat_lon

,Lat/Lon,Average,Median,Min,Max,Mode
0,Latitude,39.31,39.25,25.14,64.89,[[38.93]]
1,Longitude,-97.45,-91.65,-167.13,-68.84,"[[-122.41, -120.09, -119.98, -119.98, -77.91]]"


In [12]:
# Find most common year for sightings

# Create new data frame for years, dropping null values
years_df = pd.DataFrame(bigfoot_data["date"])
years_df = years_df.dropna()
# Converts date to year only and creates new column
years_df["year"] = pd.DatetimeIndex(years_df["date"]).year
# Summary statistics
year_min = years_df["year"].min()
year_max = years_df["year"].max()
year_mode = years_df["year"].mode()[0]
years_summary = pd.DataFrame({"Earliest Year": [year_min],
                             "Most Recent Year": [year_max],
                             "Most Seen Year": [year_mode]})
years_summary

,Earliest Year,Most Recent Year,Most Seen Year
0,1869,2018,2004


In [13]:
# Import API for gmaps just in this cell
from api_keys import g_key
import gmaps
gmaps.configure(api_key=g_key)

# Make data frame of just latitudes and longitudes
bigfoot_data_no_na = pd.DataFrame({"Latitude": bigfoot_data["latitude"].dropna(),
                                   "Longitude": bigfoot_data["longitude"].dropna()})
locations = bigfoot_data_no_na[["Latitude", "Longitude"]].astype(float)

# Create map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))